# Análisis comparativo: Secuencial, Multi-hilos y Multi-procesos - Jose Manuel Mora Balderas - 174707

En este notebook, vamos a realizar un análisis comparativo entre tres enfoques de programación:
- **Secuencial**: El programa se ejecuta de principio a fin sin utilizar concurrencia.
- **Concurrente con multi-hilos**: Se divide el trabajo en varios hilos concurrentes.
- **Concurrente con multi-procesos**: Se divide el trabajo entre varios procesos separados.

La tarea a resolver en los tres enfoques será calcular la suma de los cuadrados de los primeros **N** números enteros.
Realizaremos cada ejecución 5 veces para asegurar resultados consistentes y calcularemos la **mediana** de los tiempos de ejecución.


In [45]:
import time
import numpy as np

# Enfoque Secuencial: Calcula la suma de los cuadrados de los primeros N números enteros.
def sum_of_squares_sequential(N):
    sum = 0
    for i in range(N):
      sum = sum + i**2
    return sum

N = 10**8  # Definimos el valor de N para el experimento (100 millones)
times_sequential = []  # Almacenará los tiempos de ejecución

# Ejecutamos la función secuencial 20 veces y almacenamos los tiempos
for _ in range(5):
    start_time = time.time()
    result = sum_of_squares_sequential(N)
    end_time = time.time()
    times_sequential.append(end_time - start_time)

# Calculamos la mediana de los tiempos
median_time_sequential = np.median(times_sequential)

print(f"Mediana del tiempo secuencial: {median_time_sequential} segundos")


Mediana del tiempo secuencial: 35.45701861381531 segundos


In [46]:
import threading
import time

# Función para calcular la suma de los cuadrados en un rango específico, de manera concurrente en un hilo
def sum_of_squares(start, end, result, index):
    """
    Calcula la suma de los cuadrados de los números en el rango [start, end).
    El resultado se almacena en la lista `result` en la posición `index` para identificar el hilo.
    """
    sum = 0
    for i in range(start, end):
        sum += i**2  # Calcula el cuadrado de i y lo suma
    result[index] = sum  # Guarda el resultado de este hilo en la posición correspondiente de `result`

# Función principal para gestionar múltiples hilos y sumar los resultados
def sum_of_squares_multithread(N, num_threads=4):
    """
    Divide la tarea de calcular la suma de los cuadrados entre varios hilos.
    Cada hilo calcula una parte de la suma en un rango de números.
    """
    threads = []  # Lista para almacenar los hilos
    results = [0] * num_threads  # Lista para almacenar los resultados de cada hilo
    chunk_size = N // num_threads  # Divide el rango de números en partes iguales para cada hilo

    for i in range(num_threads):
        # Calcula los límites del rango que cada hilo procesará
        start = i * chunk_size
        end = N if i == num_threads - 1 else (i + 1) * chunk_size
        # Crea un hilo para ejecutar la función `sum_of_squares` en el rango definido
        thread = threading.Thread(target=sum_of_squares, args=(start, end, results, i))
        threads.append(thread)  # Añade el hilo a la lista
        thread.start()  # Inicia la ejecución del hilo

    # Espera a que todos los hilos terminen su ejecución
    for thread in threads:
        thread.join()

    # Retorna la suma total de todos los resultados calculados por los hilos
    return sum(results)

# Ejecutar la función multi-hilos 20 veces y calcular la mediana de los tiempos de ejecución
N = 10**8  # Definimos el valor de N para el experimento (10 millones)
times_multithreading = []  # Almacenará los tiempos de ejecución

# Ejecutamos la función multi-hilos 20 veces y almacenamos los tiempos
for _ in range(5):
    start_time = time.time()
    result = sum_of_squares_multithread(N)
    end_time = time.time()
    times_multithreading.append(end_time - start_time)

# Calculamos la mediana de los tiempos
median_time_multithreading = np.median(times_multithreading)

print(f"Mediana del tiempo secuencial: {median_time_multithreading} segundos")


Mediana del tiempo secuencial: 36.32996416091919 segundos


In [47]:
import multiprocessing
import time

# Función para calcular la suma de los cuadrados en un rango específico, ejecutada en un proceso
def sum_of_squares(start, end, queue):
    """
    Calcula la suma de los cuadrados de los números en el rango [start, end).
    El resultado se coloca en una cola para que el proceso principal lo recoja.
    """
    sum = 0
    for i in range(start, end):
        sum += i**2  # Calcula el cuadrado de i y lo suma
    queue.put(sum)  # Coloca el resultado en la cola para que el proceso principal lo recoja

# Función principal para gestionar múltiples procesos y sumar los resultados
def sum_of_squares_multiprocess(N, num_processes=4):
    """
    Divide la tarea de calcular la suma de los cuadrados entre varios procesos.
    Cada proceso calcula una parte de la suma en un rango de números.
    """
    processes = []  # Lista para almacenar los procesos
    queue = multiprocessing.Queue()  # Cola para recolectar los resultados de los procesos
    chunk_size = N // num_processes  # Divide el rango de números en partes iguales para cada proceso

    for i in range(num_processes):
        # Calcula los límites del rango que cada proceso procesará
        start = i * chunk_size
        end = N if i == num_processes - 1 else (i + 1) * chunk_size
        # Crea un proceso para ejecutar la función `sum_of_squares` en el rango definido
        process = multiprocessing.Process(target=sum_of_squares, args=(start, end, queue))
        processes.append(process)  # Añade el proceso a la lista
        process.start()  # Inicia la ejecución del proceso

    # Recolecta los resultados de la cola
    results = [queue.get() for _ in range(num_processes)]

    # Espera a que todos los procesos terminen su ejecución
    for process in processes:
        process.join()

    # Retorna la suma total de todos los resultados calculados por los procesos
    return sum(results)

# Ejecutar la función multi-procesos 20 veces y calcular la mediana de los tiempos de ejecución
N = 10**8  # Definimos el valor de N para el experimento (10 millones)
times_multiprocessing = []  # Almacenará los tiempos de ejecución

# Ejecutamos la función multi-procesos 20 veces y almacenamos los tiempos
for _ in range(5):
    start_time = time.time()
    result = sum_of_squares_multiprocess(N)
    end_time = time.time()
    times_multiprocessing.append(end_time - start_time)

# Calculamos la mediana de los tiempos
median_time_multiprocessing = np.median(times_multiprocessing)

print(f"Mediana del tiempo secuencial: {median_time_multiprocessing} segundos")


Mediana del tiempo secuencial: 35.85261106491089 segundos


Al ver los tiempos dados por cada implementacion podemos ver que el que tarda menos es el secuencial. La versión secuencial suele ser más rápida (en estos casos) porque usar hilos o procesos tiene un costo extra en organizar y manejar toda esta tarea.

Para tareas sencillas, como sumar esta de sumar cuadrados (aunque sean 10 millones de numeros), este esfuerzo adicional hace que el tiempo total sea mayor, ya que dividir el trabajo no vale la pena comparado con hacerlo de forma directa y sin dividir.

Para este ejemplo en concreto de la suma de los primeros 10 millones de cuadrados, la implementacion en correrlo secuencialmente es mas eficiente por lo anteriormente mencionado.

In [47]:
"""                                                                                                                         |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||   |^^^^^^^^^^^\||____
                                                                                                                            |||   | IMPLEMENTACION |||""'|""\
                                                                                                                            |||   | SECUENCIAL     ||        |
                                                                                                                            |||   | _______________||__|__|__|)
                                                                                                                            |||      (@)@)"""""""**(@)(@)
                                                                                                                            |||
                                                                                                                            |||
                                                                              |^^^^^^^^^^^\||____                           |||
                                                                              | IMPLEMENTACION |||""'|""\                   |||
                                                                              | MULTIHILOS     ||        |                  |||
                                                                              | _______________||__|__|__|)                 |||
                                                                                 (@)@)"""""""**(@)(@)                       |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                        |^^^^^^^^^^^\||______||
                                                                                                        | IMPLEMENTACION    |||""'|""\
                                                                                                        | MULTIPROCESOS     ||        |
                                                                                                        | __________________||__|__|__|)
                                                                                                           (@)@)"""""""**(@)(@)
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |M|
                                                                                                                            |E|
                                                                                                                            |T|
                                                                                                                            |A|
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
                                                                                                                            |||
ojala esto me gane puntos por creatividad :)                                                                                |||
"""
